In [75]:
from openseespy              import opensees as osp # Opensees interface

osp.wipe()
osp.model('basic', '-ndm', 2, '-ndf', 2)
# Nodes
osp.node(0, 0, 1)
osp.node(1, 0, 0)
osp.node(2, 1, 1)

# Element
osp.uniaxialMaterial("Elastic", 0, 500)
osp.element("Truss", 0, *(0, 2), 200, 0)
osp.element("Truss", 1, *(1, 2), 200, 0)

# Support
osp.fix(0, True, True)
osp.fix(1, True, True)

# Load
osp.timeSeries('Constant', 1)  # Define a constant time series for loading
osp.pattern("Plain", 1, 1)      # Define a plain load pattern
osp.load(2, *(0, -1))

# Define the solution procedure in OpenSees
osp.system("FullGeneral")                   # Define the system of equations
osp.numberer("RCM")                     # Define the numbering algorithm (Reverse Cuthill-McKee)
osp.constraints("Plain")                # Define the constraint handler
osp.integrator("LoadControl", 1.0)       # Define the integrator for the analysis (Load Control with step size 1.0)
osp.algorithm("Linear")                  # Define the solution algorithm (Linear)
osp.analysis("Static")                   # Define the type of analysis (Static)

# Perform the analysis
osp.analyze(1)  # Analyze one load step

0

[[1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.35355339e+05 3.53553391e+04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00 3.53553391e+04 3.53553391e+04]]


In [14]:
# Libraries and setup
import json              # Save and load JSON files
import numpy      as np  # Math library
import pandas     as pd  # Dataframe library
import pprint            # Aesthetic of output
import re                # RegEx
import sklearn           # Machine learning framework
import torch             # Neural network framework
import tqdm              # Loading bar

from datetime                import datetime        # Tools for time formatting
from openseespy              import opensees as osp # Opensees interface
from torch                   import nn              # Neural network parts
from torch                   import tensor          # Tensor
from torch.utils.data        import DataLoader      # Dataset to batch
from torch.utils.data        import Dataset         # Dataset class
from torch.utils.data        import random_split    # Split Dataset to Subsets
from torch.utils.tensorboard import SummaryWriter   # Writer
from sklearn.preprocessing   import StandardScaler  # Scaler

np.set_printoptions(threshold=np.inf)
np.set_printoptions(linewidth=np.inf)

seed = 42
torch.manual_seed(seed)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [38]:
class Linear10BarsTrussDataset(Dataset):
    """
    Dataset containing trusses relevant data.
    """
    def __init__(self, filename, nrows=None):
        self.df = pd.read_csv(filename, nrows=nrows)
        
        self.data = self.df.drop(['n_cells',
                                  'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4', 'x5', 'y5', 'x6', 'y6',
                                  'fix_x1', 'fix_y1', 'fix_x2', 'fix_y2', 'fix_x3', 'fix_y3',
                                  'fix_x4', 'fix_y4', 'fix_x5', 'fix_y5', 'fix_x6', 'fix_y6', 
                                  'P_x1', 'P_y1', 'P_x2', 'P_y2', 'P_x3', 'P_y3', 
                                  'P_x4','P_x5', 'P_y5', 'P_x6'], axis=1)
        
        # Change units
        # Area in cm^2
        exp = re.compile('A_[0..9]*')
        areas_col = [col for col in self.df.columns if exp.match(col)]
        self.df[areas_col] *= 1.0e4
        
        # Young modulus in GPa
        exp = re.compile('E_[0..9]*')
        youngs_col = [col for col in self.df.columns if exp.match(col)]
        self.df[youngs_col] *= 1.0e-9
        
        # Forces in kN
        exp = re.compile('P_[x,y][0..9]*')
        forces_col = [col for col in self.df.columns if exp.match(col)]
        self.df[forces_col] *= 1.0e-3
        
        # Identify data
        self.target = self.df[['E_1', 'E_2', 'E_3', 'E_4', 'E_5', 'E_6', 'E_7', 'E_8', 'E_9', 'E_10']]
        self.data = self.data.drop(self.target.columns, axis=1)
        
        self.data_features = self.data.columns
        self.target_features = self.target.columns
        
        self.target = tensor(self.target.to_numpy())
        self.data = tensor(self.data.to_numpy())
    
    def __len__(self):
        return self.target.__len__()

    def __getitem__(self, idx):
        return self.data[idx], self.target[idx], self.df.iloc[idx]

In [39]:
class SimpleFNN(nn.Module):
    """
    Simple Feed Forward Network
    """
    def __init__(self, size_in, size_out):
        super(SimpleFNN, self).__init__()
        self.layer_1 = nn.Linear(size_in, 100, dtype=torch.double)
        self.activ_1 = nn.ReLU()
        self.layer_2 = nn.Linear(100, size_out, dtype=torch.double)
        self.activ_2 = nn.Sigmoid()
        
    def forward(self, x):
        x = self.layer_1(x)
        x = self.activ_1(x)
        x = self.layer_2(x)
        x = self.activ_2(x)
        return x

In [40]:
#%%capture
# Reading dataset
N_EPOCH = 100
BATCH_SIZE = 64

data_dir = "./dataset_general_100_000/"
with open(f"{data_dir}/info.json") as f:
    _info = json.load(f)
pprint.pprint(_info, width=150)

ds = Linear10BarsTrussDataset(f"{data_dir}/data.csv", nrows=100_000)
train_ds, test_ds, eval_ds = random_split(ds, [.7,.2,.1])

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
eval_dl = DataLoader(eval_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)

{'date': '20/10/2024 11:40:14',
 'model': 'LinearTrussGenerator',
 'model_arguments': {'distributions': {'length': {'parameters': [4.0, 10.0], 'type': 'UNIFORM'},
                                       'load': {'parameters': [-1000000.0, -1000.0], 'type': 'UNIFORM'}},
                     'name': "Linear cantilever's cross pattern beam",
                     'parameters': {'areas': {'default': {'parameters': [0.001, 0.1], 'type': 'UNIFORM_CONST'}},
                                    'cell_height': {'default': {'parameters': ['length'], 'type': 'DISTRIBUTION'}},
                                    'cell_length': {'default': {'parameters': ['length'], 'type': 'DISTRIBUTION'}},
                                    'cell_number': {'default': {'parameters': [2], 'type': 'CONSTANT'}},
                                    'loads': {'4-y': {'parameters': ['load'], 'type': 'DISTRIBUTION'},
                                              '6-y': {'parameters': ['load'], 'type': 'DISTRIBUTION'},
    

In [47]:
_,_,df = ds[0:10]
df

,n_cells,cell_height,cell_length,x1,y1,x2,y2,x3,y3,x4,...,N_1,N_2,N_3,N_4,N_5,N_6,N_7,N_8,N_9,N_10
0,2,8.684038,8.684038,0.0,4.342019,0.0,-4.342019,8.684038,4.342019,8.684038,...,1238769.20,254422.060,-1297548.60,-379657.400,225032.390,254422.060,938287.060,-855160.50,536916.700,-359807.120
1,2,8.975055,8.975055,0.0,4.487527,0.0,-4.487527,8.975055,4.487527,8.975055,...,1706282.20,350441.250,-1787245.00,-522940.560,309959.880,350441.250,1292397.600,-1177899.10,739549.600,-495598.750
2,2,6.651061,6.651061,0.0,3.325531,0.0,-3.325531,6.651061,3.325531,6.651061,...,820711.30,168560.080,-859653.90,-251531.200,149088.800,168560.080,621635.400,-566562.25,355718.840,-238379.950
3,2,4.829182,4.829182,0.0,2.414591,0.0,-2.414591,4.829182,2.414591,4.829182,...,1261558.90,259102.660,-1321419.60,-386641.970,229172.310,259102.660,955548.700,-870892.94,546794.300,-366426.500
4,2,4.596314,4.596314,0.0,2.298157,0.0,-2.298157,4.596314,2.298157,4.596314,...,290618.22,59688.016,-304407.97,-89068.530,52793.133,59688.016,220124.360,-200622.69,125961.920,-84411.600
5,2,7.067292,7.067292,0.0,3.533646,0.0,-3.533646,7.067292,3.533646,7.067292,...,293969.90,60376.400,-307918.72,-90095.760,53401.996,60376.400,222663.060,-202936.47,127414.640,-85385.125
6,2,9.169996,9.169996,0.0,4.584998,0.0,-4.584998,9.169996,4.584998,9.169996,...,1938310.60,398095.900,-2030283.00,-594052.500,352109.700,398095.900,1468144.000,-1338075.40,840117.100,-562992.600
7,2,9.806996,9.806996,0.0,4.903498,0.0,-4.903498,9.806996,4.903498,9.806996,...,309090.70,63481.950,-323756.97,-94729.960,56148.805,63481.950,234116.060,-213374.80,133968.400,-89777.030
8,2,4.362812,4.362812,0.0,2.181406,0.0,-2.181406,4.362812,2.181406,4.362812,...,1271342.00,261111.920,-1331666.90,-389640.280,230949.480,261111.920,962958.750,-877646.44,551034.560,-369268.030
9,2,5.145792,5.145792,0.0,2.572896,0.0,-2.572896,5.145792,2.572896,5.145792,...,163811.42,33644.070,-171584.23,-50204.848,29757.662,33644.070,124076.484,-113084.06,71000.375,-47579.900


In [100]:
def compute_residual(df):
    n_cells = df['n_cells']
    n_bars  = 5*n_cells
    n_nodes = 2*(n_cells + 1)
    
    # Get the nodes
    exp_x = re.compile('x[0..9]*')
    exp_y = re.compile('y[0..9]*')
    
    x_cols = [col for col in df.columns if exp_x.match(col)]
    y_cols = [col for col in df.columns if exp_y.match(col)]
    
    x = [df.iloc[i][x_cols].to_numpy() for i in df.index]
    y = [df.iloc[i][y_cols].to_numpy() for i in df.index]
    
    nodes = np.stack((x,y), axis=2)
    
    return nodes

nodes = compute_residual(df)
nodes.shape

(10, 6, 2)

In [5]:
# Model and
model = SimpleFNN(len(ds.data_features), len(ds.target_features))
model.to(device)

loss_fn  = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

In [6]:
# Train scalers
x_scaler = StandardScaler()
y_scaler = StandardScaler()

for x, y in train_dl:
    x_scaler.partial_fit(x)
    y_scaler.partial_fit(y)

In [7]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter(f"runs/LinearTrussFNN_{timestamp}")
best_loss = np.infty
best_test_loss = np.infty
mae_fn = nn.L1Loss()
for epoch in range(N_EPOCH):
    print('EPOCH {}:'.format(epoch + 1))
    
    model.train(True)
    
    running_train_loss = 0.
    running_train_mae = 0
    running_len = 0
    last_loss = np.infty
    for i, batch in enumerate(train_dl):
        running_len += 1
        x, y, _ = batch
        x = tensor(x_scaler.transform(x), device=device)
        y = tensor(y_scaler.transform(y), device=device)
        
        optimizer.zero_grad()
        y_pred = model(x)

        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()
        
        mae = mae_fn(tensor(y_scaler.inverse_transform(y_pred.cpu().detach().numpy())),
                     tensor(y_scaler.inverse_transform(y.cpu().detach().numpy())))
        
        running_train_loss += loss.item()
        running_train_mae += mae.item()
        
        if running_len == 100 or (i+1) == len(train_dl):
            last_loss = running_train_loss / running_len # loss per batch
            last_mae = running_train_mae / running_len
            
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch * len(train_dl) + i + 1
            writer.add_scalar('MSE/train', last_loss, tb_x)
            writer.add_scalar('MAE/train', last_mae, tb_x)
            running_train_loss = 0
            running_train_mae = 0                  
            running_len = 0

    avg_train_loss = last_loss
    avg_train_mae = last_mae
    
    running_test_loss = 0
    running_test_mae = 0
    model.eval()

    with torch.no_grad():
        for i, batch in enumerate(test_dl):  # Use DataLoader to load test data in batches
            x, y, _ = batch
            x = tensor(x_scaler.transform(x), device=device)
            y = tensor(y_scaler.transform(y), device=device)
            
            y_pred = model(x)
            test_loss = loss_fn(y_pred, y)
            running_test_loss += test_loss.item()

            mae = mae_fn(tensor(y_scaler.inverse_transform(y_pred.cpu().detach().numpy())),
                         tensor(y_scaler.inverse_transform(y.cpu().detach().numpy())))

            running_test_mae += mae.item()

    avg_test_loss = running_test_loss / len(test_dl)  # Divide by number of batches
    avg_test_mae = running_test_mae / len(test_dl)  # Divide by number of batches
    print('LOSS train {} valid {}'.format(avg_train_loss, avg_test_loss))
    print('MAE train {} valid {}'.format(avg_train_mae, avg_test_mae))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalar('MSE/validation', avg_test_loss, epoch)
    writer.add_scalar('MAE/validation', avg_test_mae, epoch)
    
    #writer.add_scalars('Training vs. Validation Loss',
    #                { 'Training' : avg_train_loss, 'Validation' : avg_test_loss },
    #                epoch + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_test_loss < best_test_loss:
        best_test_loss = avg_test_loss
        model_path = 'model_{}_{}'.format(timestamp, epoch)
        torch.save(model.state_dict(), model_path)
    

EPOCH 1:
  batch 100 loss: 1.0672617891241887
  batch 200 loss: 1.0037822532476888
  batch 300 loss: 1.0140957894402662
  batch 400 loss: 0.9697347056287203
  batch 500 loss: 0.9956144703234452
  batch 600 loss: 0.9875832379665869
  batch 700 loss: 0.9437476293772227
  batch 800 loss: 0.9500484896568991
  batch 900 loss: 0.8901252799573329
  batch 1000 loss: 0.8761901750983107
  batch 1094 loss: 0.8617039838665395
LOSS train 0.8617039838665395 valid 0.8461031369544714
MAE train 11.369495102532236 valid 11.242415594876578
EPOCH 2:
  batch 100 loss: 0.8313130605814671
  batch 200 loss: 0.8345266745210086
  batch 300 loss: 0.8302688006760041
  batch 400 loss: 0.8146787133131999
  batch 500 loss: 0.7809165768670955
  batch 600 loss: 0.7697037548227584
  batch 700 loss: 0.7839069336504422
  batch 800 loss: 0.7439119638460793
  batch 900 loss: 0.7483282525313176
  batch 1000 loss: 0.7664178276022219
  batch 1094 loss: 0.7355735289701867
LOSS train 0.7355735289701867 valid 0.736297404627205
M